In [1]:
# import sys
# !{sys.executable} -m pip install ../. --quiet --user #installation of library in local enviorment
from pyRN import pyRN
from bitarray import bitarray as bt
from bitarray import frozenbitarray as fbt
import numpy as np

In [2]:
file="../networks/op_test_net.txt"
RN=pyRN.setFromText(file)
RN.SpIdStrArray

array(['s1', 's2', 's3', 'x1', 'x2'], dtype=object)

Once the libraries and the network are loaded, we can obtain the overproducible species of a network by means of the function `getOpSp()`, the latter receives as arguments the set of present species `sp_set` and the present reaction set `pr`. It considers the reactions triggered by such present species intersected by the considered reaction `pr`. If `pr` is given as `None` all reaction will be considered feasible. Here is how it is used:

In [3]:
print("The whole reaction network is:")
RN.printRp()

# Here we obtain the overprducible species of the network
opsp=RN.getallOpSpBt(sp_set=RN.SpIdStrArray,pr=None)
print("In this reaction network, the species",RN.SpIdStrArray[opsp.search(1)],"are overproducible")

sp=RN.SpIdStrArray[[0,1,3]]
print("By selecting the species",sp," we reduce the reaction network to the triggerable reactions:")
RN.printRp(RN.getTriggerableRpBtFromSp(sp))

opsp=RN.getallOpSpBt(sp_set=RN.SpIdStrArray[[0,1,3]],pr=None)
print("So now the overproducible species are:",RN.SpIdStrArray[opsp.search(1)])

pr=np.array([1,2])
print("We can also reduce the set to reaction only considering", pr, "to occur:")
RN.printRp(pr)
opsp=RN.getallOpSpBt(sp_set=RN.SpIdStrArray[[0,1,3]],pr=pr)
print("by this,",RN.SpIdStrArray[opsp.search(1)],"are overproducible")


The whole reaction network is:
r0:   s1 => s2 
r1:   s2 + x1 => s1 
r2:   x1 => 2s1 + 2x1 
r3:   x1 => x2 
r4:   x2 => s3 
r5:   2s3 => x2 
In this reaction network, the species ['s1' 's2' 's3' 'x1' 'x2'] are overproducible
By selecting the species ['s1' 's2' 'x1']  we reduce the reaction network to the triggerable reactions:
r0:   s1 => s2 
r1:   s2 + x1 => s1 
r2:   x1 => 2s1 + 2x1 
r3:   x1 => x2 
So now the overproducible species are: ['s1' 's2' 'x1']
We can also reduce the set to reaction only considering [1 2] to occur:
r1:   s2 + x1 => s1 
r2:   x1 => 2s1 + 2x1 
by this, ['s1' 'x1'] are overproducible


Given a set of species `sp_set` and a set of feasible reactions `pr`, such are not an organization. It is possible to obtain a set of reactions added as inflow of the system *i.e.* $\emptyset \to s$, such that the new system (with added reactions) become an organization. Thus the function `getSpNeededToOrg` corresponds to a vector of indices of the vector `RN.SpIdStrArray`, species that must be added as inflow. The second component of the tuple, corresponds to a vector of indexes of reactions that are active. 

In [4]:
sp=bt(len(RN.SpIdStrArray))
sp.setall(0)
sp[0:2]=1
is_sm=RN.isSmFromSp(sp)

print("The set of species:",RN.SpIdStrArray[sp.search(1)],"is self-maintained?",is_sm)
result=RN.getSpNeededToOrg(sp_set=sp,pr=None)
print("We need the specie(s):",RN.SpIdStrArray[result[0]],", added as inflow reaction(s) to become an organization.")
print("whose reaction network corresponds to:")
RN.printRp(result[1])

The set of species: ['s1' 's2'] is self-maintained? False
We need the specie(s): ['s1'] , added as inflow reaction(s) to become an organization.
whose reaction network corresponds to:
r0:   s1 => s2 


We can repeat this but selecting which reaction cab be activiating via the `pr`.

In [5]:
sp=bt(len(RN.SpIdStrArray))
sp.setall(0)
sp[2:5]=1
is_sm=RN.isSmFromSp(sp)
pr=[3,4,5]
print("The set of species:",RN.SpIdStrArray[sp.search(1)],"is self-maintained?",is_sm)
result=RN.getSpNeededToOrg(sp_set=sp,pr=pr)
print("We need the specie(s):",RN.SpIdStrArray[result[0]],", added as inflow reaction(s) to become an organization.")
print("whose reaction network corresponds to:")
RN.printRp(result[1])

The set of species: ['s3' 'x1' 'x2'] is self-maintained? True
We need the specie(s): ['s2'] , added as inflow reaction(s) to become an organization.
whose reaction network corresponds to:
r3:   x1 => x2 
r4:   x2 => s3 
r5:   2s3 => x2 


If we consist of an organization and its set of overproducible species, we can ask if any subset of these can be overproduced. It is worth noting, that not always some subset of overproducible(s), can be overproduced. The function `veriOpSp` do this verification operation. It receives as argument the species of the `sp_set` organization, `op_set` overproducible species to be verified, the present species `pr` and finally a bool value  `force_org` . This last variable imposes that all the `pr` reactions have their component of the process vector greater than zero. Thus the overproduction is conditional on sp_set being maintained as an organization.   

In [6]:
sp=bt(len(RN.SpIdStrArray))
sp.setall(0)
sp[1:3]=1 #selecting the species we want to be overpoduced

print("Are the species",RN.SpIdStrArray[sp.search(1)],"overproducible in the organization",RN.SpIdStrArray,"?")
verification, process = RN.veriOpSpBt(sp_set=RN.SpIdStrArray,opsp_set=sp,pr=None,force_org=False)
print(verification,",by means of the process:",process)

Are the species ['s2' 's3'] overproducible in the organization ['s1' 's2' 's3' 'x1' 'x2'] ?
True ,by means of the process: [2. 0. 1. 1. 1. 0.]


We note that the resulting process vector does not trigger all reactions, thus such a process does not guarantee that the analyzed set is an organization. However, the resulting process output will not consume any species. by using the option `force_org=True`, allows us to find a process vector that has all its components greater than zero (all active).

In [7]:
sp=bt(len(RN.SpIdStrArray))
sp.setall(0)
sp[1:3]=1 #selecting the species we want to be overpoduced

print("Are the species",RN.SpIdStrArray[sp.search(1)],"overproducible in the organization",RN.SpIdStrArray,"?")
verification, process = RN.veriOpSpBt(sp_set=RN.SpIdStrArray,opsp_set=sp,pr=None,force_org=True) #We change the force_org to True
print(verification,",by means of the process:",process)

Are the species ['s2' 's3'] overproducible in the organization ['s1' 's2' 's3' 'x1' 'x2'] ?
True ,by means of the process: [7. 1. 3. 2. 3. 1.]


It should be noted that there are subsets of overproduced species, which may not necessarily be overproduced:

In [8]:
sp=bt(len(RN.SpIdStrArray))
sp.setall(0)
sp[2]=1 #selecting the species we want to be overpoduced

print("Is the specie",RN.SpIdStrArray[sp.search(1)],"overproducible in the organization",RN.SpIdStrArray,"?")
verification, process = RN.veriOpSpBt(sp_set=RN.SpIdStrArray,opsp_set=sp,pr=None,force_org=False)
print(verification)

Is the specie ['s3'] overproducible in the organization ['s1' 's2' 's3' 'x1' 'x2'] ?
False


Or we can consider only a subset of reactions by using the `pr`

In [9]:
sp=bt(len(RN.SpIdStrArray))
sp.setall(0)
sp[2:5]=1 # selecting the species we want to be overpoduced
pr=[3,4,5] # also subsetting a reactions

print("Are the species",RN.SpIdStrArray[sp.search(1)],"overproducible in the orgnaization",RN.SpIdStrArray,"activating only the following reactions?")
RN.printRp(pr)
verification, process = RN.veriOpSpBt(sp_set=RN.SpIdStrArray,opsp_set=sp,pr=pr,force_org=False)
print(verification)

Are the species ['s3' 'x1' 'x2'] overproducible in the orgnaization ['s1' 's2' 's3' 'x1' 'x2'] activating only the following reactions?
r3:   x1 => x2 
r4:   x2 => s3 
r5:   2s3 => x2 
False


Once in possession of the process vector that overlays a subset of the organization in question. We can identify the dynamic roles that have models (subsets) operating under this process. The decomposition allows this identification. Four types of modules with distinct roles are identified the organization in four types of roles:

- F, the set of overproduced
- E, the set of catalysts
- C_i, the collection of fragile cycles.
- N_r, the set of non-reactive reactions.

The fragile cycles correspond to modules that are self-maintained within the network. We will not go into detail, but we can mention that they are dynamically directly connected to catalysts and spbreproduced, and not to each other. Depending on the species they produce, they can be critical in the organizational structure if disturbed. 

By use of function `getDcomArray` we can obtain an array of the decomposition for a given organization an process vector. It recives as input the species of the set (`sp_set`), the set of the overproduced species (`opsp_set`) and the present species set `pr`. It returns an array whose components indicate the  function performed by each species, correlative positions to sp species  vector. If the value is `-1` it corresponds to an overproducible species, if it is `-2` to a catalytic species. `-3`if its a non-reactive species and if it is 0 the species is not present. The integer values indicate ot which fragile cycle belogns.

In [10]:
sp=bt(len(RN.SpIdStrArray))
sp.setall(0)
sp[1]=1 #selecting the species we want to be overpoduced

print("Are the species",RN.SpIdStrArray[sp.search(1)],"overproducible in the organization",RN.SpIdStrArray,"?")
verification, process = RN.veriOpSpBt(sp_set=RN.SpIdStrArray,opsp_set=sp,pr=None,force_org=True)
print(verification,",by means of the process:",process)
RN.printRpFromProcess(np.where(process>=0)[0],process)
print("the latter process and orgaization result in the decomposition:")
decom=RN.getDcomArray(sp_set=RN.SpIdStrArray,opsp_set=sp,pr=np.where(process)[0])
print(decom)

Are the species ['s2'] overproducible in the organization ['s1' 's2' 's3' 'x1' 'x2'] ?
True ,by means of the process: [5. 1. 2. 1. 2. 1.]
r0:   s1 (5.0) => s2 
r1:   s2 + x1 (1.0) => s1 
r2:   x1 (2.0) => 2s1 + 2x1 
r3:   x1 (1.0) => x2 
r4:   x2 (2.0) => s3 
r5:   2s3 (1.0) => x2 
the latter process and orgaization result in the decomposition:
[ 1. -1.  1.  1.  1.]


In [11]:
sp=bt(len(RN.SpIdStrArray))
sp.setall(0)
sp[1]=1 #selecting the species we want to be overpoduced

print("Are the species",RN.SpIdStrArray[sp.search(1)],"overproducible in the organization",RN.SpIdStrArray,"?")
verification, process = RN.veriOpSpBt(sp_set=RN.SpIdStrArray,opsp_set=sp,pr=None,force_org=False)
print(verification,",by means of the process:",process)
RN.printRpFromProcess(np.where(process>=0)[0],process)
print("the latter process and orgaization result in the decomposition:")
decom=RN.getDcomArray(sp_set=RN.SpIdStrArray,opsp_set=sp,pr=np.where(process)[0])
print(decom)

Are the species ['s2'] overproducible in the organization ['s1' 's2' 's3' 'x1' 'x2'] ?
True ,by means of the process: [1.5 0.5 0.5 0.  0.  0. ]
r0:   s1 (1.5) => s2 
r1:   s2 + x1 (0.5) => s1 
r2:   x1 (0.5) => 2s1 + 2x1 
r3:   x1 (0.0) => x2 
r4:   x2 (0.0) => s3 
r5:   2s3 (0.0) => x2 
the latter process and orgaization result in the decomposition:
[ 1. -1. -3.  1. -3.]


We can appreciate that the decomposition is sensitive to the process vector. If we select a porcess vector were some componets are zero, it can result un non-reactive components (-3) of the decomposition. 

In consideration of the overproduced species, there is a collection whose elements are the smallest set of species that can be overproduced. This collection constitutes a basis given the additive property of the processes that overproduce these sets.
The function `getOpBase` generates a list of this base, receiving as input the same parameters as the previous functions but not considering the overproduced species. It returns (`op_base`) a `networkx` directed graph, representing a Hasse diagram. Where each node represents an overproduced subset. The edges correspond to direct (lattice) contentions. Each node also has the following properties:

- `level`: the number of overproduced species
- `process`: related process for the especific overpduction  (relative components pr)
- `decomposition`: decomposition obtained from the `getDecomArray `function.

In [12]:
op_base=RN.genOpBase(sp_set=RN.SpIdStrArray,pr=None,force_org=True) #We generate the overprodiuced base
for i in op_base.nodes(data=True):
    print("The organization",RN.SpIdStrArray,"overproduce the species",RN.SpIdStrArray[i[0].search(1)],"with level",i[1]['level'])
    print("under the process",i[1]['process'],", resulting in the decomposition",i[1]['decomposition'])

The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s1'] with level 1
under the process [1. 1. 2. 1. 2. 1.] , resulting in the decomposition [-1.  1.  1.  1.  1.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s2'] with level 1
under the process [5. 1. 2. 1. 2. 1.] , resulting in the decomposition [ 1. -1.  1.  1.  1.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s1' 's3'] with level 2
under the process [1. 1. 3. 2. 3. 1.] , resulting in the decomposition [-1.  1. -1.  1.  1.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s1' 'x1'] with level 2
under the process [1. 1. 3. 1. 2. 1.] , resulting in the decomposition [-1.  1.  2. -1.  2.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s1' 'x2'] with level 2
under the process [1. 1. 3. 2. 2. 1.] , resulting in the decomposition [-1.  1.  2.  1. -1.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s2' 's3'] 

It is worth noteing in the resulting collection of overproduced nodes, corresponds to a base. Therefore, all nodes whit same level have a non empty intersection between them. Also, the union of all nodes whit in a level, must present species different wiht the union of all nodes of lower levels.

Finally, given the additive possibility of the elements of the basal overproducible collection, it is possible to generate a hasse of them by means of the function `getOpHasseNx`.The input corresponds to a set of `sp_set` species which must be an organization, it also recives the argumento `force_org` as the latter functions. The output are the complemetario node made by the joint operation of `op_base`.

In [13]:
op_hasse=RN.getOpHasseNx(RN.SpIdStrArray,force_org=True)
for i in op_hasse.nodes(data=True):
    print("The organization",RN.SpIdStrArray,"overproduce the species",RN.SpIdStrArray[i[0].search(1)],"with level",i[1]['level'])
    print("under the process",i[1]['process'],", resulting in the decomposition",i[1]['decomposition'])

The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s1'] with level 1
under the process [1. 1. 2. 1. 2. 1.] , resulting in the decomposition [-1.  1.  1.  1.  1.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s2'] with level 1
under the process [5. 1. 2. 1. 2. 1.] , resulting in the decomposition [ 1. -1.  1.  1.  1.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s1' 's3'] with level 2
under the process [1. 1. 3. 2. 3. 1.] , resulting in the decomposition [-1.  1. -1.  1.  1.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s1' 'x1'] with level 2
under the process [1. 1. 3. 1. 2. 1.] , resulting in the decomposition [-1.  1.  2. -1.  2.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s1' 'x2'] with level 2
under the process [1. 1. 3. 2. 2. 1.] , resulting in the decomposition [-1.  1.  2.  1. -1.]
The organization ['s1' 's2' 's3' 'x1' 'x2'] overproduce the species ['s2' 's3'] 